In [0]:
tiers = ['bronze', 'silver', 'gold']
adls_path = {tier : f'abfss://{tier}@earthquakedl.dfs.core.windows.net/' for tier in tiers}

bronze_adls = adls_path['bronze']
silver_adls = adls_path['silver']
gold_adls = adls_path['gold']

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[]

In [0]:
import requests
import json
from datetime import timedelta, date

In [0]:
#get the base parameters
dbutils.widgets.text('start_date', '')
dbutils.widgets.text('end_date', '')

start_date = dbutils.widgets.get('start_date')
end_date = dbutils.widgets.get('end_date')

In [0]:
url=f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

In [0]:
url

'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2025-04-07&endtime=2025-04-08'

In [0]:
try:
    response = requests.get(url)
    response.raise_for_status()   #raise HTTPerror for bad response

    data = response.json().get('features', [])

    if not data:
        print('No data returned for the specified data range')
    else:
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f'Data successfully saved to {file_path}')

except requests.exceptions.HTTPError as e:
    print(f'Error fetching data from API {e}')



Wrote 307513 bytes.
Data successfully saved to abfss://bronze@earthquakedl.dfs.core.windows.net//2025-04-07_earthquake_data.json


In [0]:
data[1]

{'type': 'Feature',
 'properties': {'mag': 1.7,
  'place': '55 km SSW of Whites City, New Mexico',
  'time': 1744070187481,
  'updated': 1744071798024,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2025gvfr',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=tx2025gvfr&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 44,
  'net': 'tx',
  'code': '2025gvfr',
  'ids': ',tx2025gvfr,',
  'sources': ',tx,',
  'types': ',origin,phase-data,',
  'nst': 8,
  'dmin': 0,
  'rms': 0.3,
  'gap': 139,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.7 - 55 km SSW of Whites City, New Mexico'},
 'geometry': {'type': 'Point', 'coordinates': [-104.492, 31.685, 9.0826]},
 'id': 'tx2025gvfr'}

In [0]:
output_data = {
    'start_date': start_date,
    'end_date': end_date,
    'bronze_adls': bronze_adls,
    'silver_adls': silver_adls,
    'gold_adls': gold_adls
}

output_json = json.dumps(output_data)

print(f'Serialized JSON: {output_json}')

dbutils.notebook.exit(output_json)